# 1D Mirror Detection Learner
**Goal**: correctly identify whether an abstract mirror object is within the vision range

In [5]:
import tensorflow as tf
import numpy

## Define constants

In [154]:
vision_range = 4
category_count = 6
class_count = 2  # mirror or not
sample_count = {
    "training": 8000,
    "test": 1300,
    "validation": 700
    }
learning_rate = 0.5

## Auxiliary functions

### Show result

In [79]:
def show_result(x):
    """Print the result"""
    print("{:.0f}%".format(x * 100))

### Automaticaly generate data sets

In [44]:
def random_mirror(vision_range, category_count):
    """Return a n-array with either a randomly placed mirror
        or no mirror at all.
    
    Args:
        vision_range (int): how many objects to be seen at a time
        category_count (int): total number of different objects
            including the mirror type
    Return:
        2-tuple:
             [0]: an array of size same as the "vision_range".
             [1]: 1 one-hot 2-array [1, 0] (has mirror) or [0, 1] (no mirror).
    """
    def label(x):
        if x is True:
            return [1, 0]
        else:
            return [0, 1]
    p1 = numpy.random.randint(1, high=category_count, size=vision_range-1)
    p2 = numpy.random.randint(0, category_count, 1)
    data = numpy.random.permutation(numpy.concatenate((p1, p2)))
    return (data, label(0 in data))



In [45]:
def mirror_data(n, vision_range, category_count):
    """Return a n x size matrix
    Args:
        n (int): number of data points
        vision_range (int): number of objects to be seen at a time
        category_count (int): total number of different object categories
            including the mirror
    Returns:
        A dictionary:
            data: (n, v)-sized 2D numpy array where v is the vision_range
            labels: (n,)-sized 1D numpy array (each element is either 1 or 0)
    """
    raw = [random_mirror(vision_range, category_count) for i in range(n)]
    return {
        "data": numpy.array([x[0] for x in raw]),
        "labels": numpy.array([x[1] for x in raw])
    }

## Dataset

In [155]:
dataset = {
    "training": mirror_data(sample_count['training'], vision_range, category_count),
    "test": mirror_data(sample_count['test'], vision_range, category_count),
    "validation": mirror_data(sample_count['validation'], vision_range, category_count)
}

# Memory allocation

In [156]:
x = tf.placeholder(tf.float32, [None, vision_range])

In [157]:
W = tf.Variable(tf.zeros([vision_range, class_count]))

In [158]:
b = tf.Variable(tf.zeros([class_count]))

In [159]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [160]:
y_ = tf.placeholder(tf.float32, [None, class_count])

In [161]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
)

In [162]:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [163]:
# create a session
session = tf.InteractiveSession()

In [164]:
tf.global_variables_initializer().run()

# I. Regular Neural Net

## Training



In [165]:
session.run(train_step, feed_dict={x: dataset['training']['data'], y_: dataset['training']['labels']})

In [166]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

In [167]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Test

In [168]:
test_result = session.run(accuracy, feed_dict={x: dataset['test']['data'], y_: dataset['test']['labels']})

In [169]:
show_result(test_result)

84%


## Validation

In [170]:
exam_result = session.run(accuracy, feed_dict={x: dataset['validation']['data'], y_: dataset['validation']['labels']})

In [171]:
show_result(exam_result)

83%


# II. Convolutionary Neural Net

## Weight Initialization

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


In [ ]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

In [ ]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1],
                         padding="SAME")

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])

In [ ]:
b_conv1 = bias_variable([32])

In [ ]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) +  b_conv1)

In [ ]:
h_pool1 = max_pool_2x2(h_conv1)

## Second Convolutional Layer

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])

In [ ]:
b_conv2 = bias_variable([64])

In [ ]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) +  b_conv2)

In [ ]:
h_pool2 = max_pool_2x2(h_conv2)